# Final Project ETL
    CIS 9440-UWA
    Final Project Milestone 3
    Group Number - 19


## 1. Install and import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account
import pyarrow

## 2. Load data from NYC and NY OPEN DATA Socrata API

####     2.a Load Data: NYPD Complaint Data Current (Year To Date)

In [2]:
client = Socrata("data.cityofnewyork.us",
                  "uF1U2FVkIhXT0q0AIiHkRjYEB",)
# load all the results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("5uac-w243", limit=450000)#enter your own app token

# Convert to pandas DataFrame
NYPD_COMPL = pd.DataFrame.from_records(results)

####     2.b Load Data: New York State Statewide COVID-19 Testing

In [3]:
client = Socrata("health.data.ny.gov", 
                 "uF1U2FVkIhXT0q0AIiHkRjYEB",)#enter your own app token
# load all the results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("xdss-u53e", limit=30000)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# Convert to pandas DataFrame
NYS_COV19 = pd.DataFrame.from_records(results)

In [4]:
NYPD_COMPL.head()

,cmplnt_num,addr_pct_cd,cmplnt_fr_dt,cmplnt_fr_tm,crm_atpt_cptd_cd,juris_desc,ky_cd,law_cat_cd,loc_of_occur_desc,ofns_desc,...,susp_age_group,susp_race,susp_sex,cmplnt_to_dt,cmplnt_to_tm,housing_psa,hadevelopt,station_name,transit_district,parks_nm
0,885776788,66,2020-12-23T00:00:00.000,19:50:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,350637195,77,2020-12-21T00:00:00.000,01:10:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,347843168,43,2020-11-22T00:00:00.000,22:00:00,COMPLETED,N.Y. POLICE DEPT,104,FELONY,NaN,RAPE,...,UNKNOWN,UNKNOWN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,197941396,47,2020-11-22T00:00:00.000,09:50:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,25-44,BLACK,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,298404927,25,2020-11-21T00:00:00.000,15:38:00,COMPLETED,N.Y. HOUSING POLICE,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
NYPD_COMPL.shape

(413412, 41)

In [6]:
NYS_COV19.head()

,test_date,county,new_positives,cumulative_number_of_positives,total_number_of_tests,cumulative_number_of_tests
0,2020-03-01T00:00:00.000,Albany,0,0,0,0
1,2020-03-02T00:00:00.000,Albany,0,0,0,0
2,2020-03-03T00:00:00.000,Albany,0,0,0,0
3,2020-03-04T00:00:00.000,Albany,0,0,0,0
4,2020-03-05T00:00:00.000,Albany,0,0,3,3


In [7]:
NYS_COV19.shape

(25854, 6)

## 3. Extract data needed for our project

####     3.a Extract data from NYPD Complaint Data Current (Year To Date)

In [8]:
NYPD_COMPL.columns

Index(['cmplnt_num', 'addr_pct_cd', 'cmplnt_fr_dt', 'cmplnt_fr_tm',
       'crm_atpt_cptd_cd', 'juris_desc', 'ky_cd', 'law_cat_cd',
       'loc_of_occur_desc', 'ofns_desc', 'rpt_dt', 'vic_age_group', 'vic_race',
       'vic_sex', 'x_coord_cd', 'y_coord_cd', 'latitude', 'longitude',
       'lat_lon', 'geocoded_column', ':@computed_region_efsh_h5xi',
       ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
       ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih', 'boro_nm',
       'jurisdiction_code', 'patrol_boro', 'pd_cd', 'pd_desc', 'prem_typ_desc',
       'susp_age_group', 'susp_race', 'susp_sex', 'cmplnt_to_dt',
       'cmplnt_to_tm', 'housing_psa', 'hadevelopt', 'station_name',
       'transit_district', 'parks_nm'],
      dtype='object')

In [9]:
NYPD_COMPL_df = NYPD_COMPL[["cmplnt_fr_dt","boro_nm","law_cat_cd","ofns_desc","susp_age_group",
                            "susp_race","susp_sex","vic_age_group","vic_race","vic_sex"]].copy()
NYPD_COMPL_df.head()

,cmplnt_fr_dt,boro_nm,law_cat_cd,ofns_desc,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
0,2020-12-23T00:00:00.000,NaN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,18-24,BLACK,M
1,2020-12-21T00:00:00.000,NaN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,25-44,BLACK,M
2,2020-11-22T00:00:00.000,BRONX,FELONY,RAPE,UNKNOWN,UNKNOWN,U,25-44,BLACK,F
3,2020-11-22T00:00:00.000,NaN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,25-44,BLACK,M,25-44,BLACK,F
4,2020-11-21T00:00:00.000,NaN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,18-24,BLACK HISPANIC,M


####     3.b  Extract data from New York State Statewide COVID-19 Testing

In [10]:
NYS_COV19.columns

Index(['test_date', 'county', 'new_positives',
       'cumulative_number_of_positives', 'total_number_of_tests',
       'cumulative_number_of_tests'],
      dtype='object')

In [11]:
NYS_COV19_df = NYS_COV19[["test_date","county","new_positives","total_number_of_tests"]].copy()
NYS_COV19_df.head()

,test_date,county,new_positives,total_number_of_tests
0,2020-03-01T00:00:00.000,Albany,0,0
1,2020-03-02T00:00:00.000,Albany,0,0
2,2020-03-03T00:00:00.000,Albany,0,0
3,2020-03-04T00:00:00.000,Albany,0,0
4,2020-03-05T00:00:00.000,Albany,0,3


## 4. Data Cleansing 

####   4.a Data cleansing on  NYPD Complaint Data Current (Year To Date)

In [12]:
NYPD_COMPL_df.dropna(inplace=True)
NYPD_COMPL_df.shape

(318265, 10)

In [13]:
NYPD_COMPL_df.isna().sum()

cmplnt_fr_dt      0
boro_nm           0
law_cat_cd        0
ofns_desc         0
susp_age_group    0
susp_race         0
susp_sex          0
vic_age_group     0
vic_race          0
vic_sex           0
dtype: int64

In [14]:
NYPD_COMPL_df.vic_race.value_counts()

BLACK                             87548
UNKNOWN                           81332
WHITE HISPANIC                    61267
WHITE                             48210
ASIAN / PACIFIC ISLANDER          24271
BLACK HISPANIC                    14591
AMERICAN INDIAN/ALASKAN NATIVE     1046
Name: vic_race, dtype: int64

In [15]:
NYPD_COMPL_df.boro_nm.value_counts()

BROOKLYN         87971
MANHATTAN        80547
BRONX            71563
QUEENS           66558
STATEN ISLAND    11626
Name: boro_nm, dtype: int64

In [16]:
NYPD_COMPL_df.boro_nm.unique()

array(['BRONX', 'QUEENS', 'MANHATTAN', 'BROOKLYN', 'STATEN ISLAND'],
      dtype=object)

In [17]:
NYPD_COMPL_df.susp_race.value_counts()

BLACK                             115077
UNKNOWN                            96059
WHITE HISPANIC                     50569
WHITE                              29140
BLACK HISPANIC                     15946
ASIAN / PACIFIC ISLANDER           10855
AMERICAN INDIAN/ALASKAN NATIVE       619
Name: susp_race, dtype: int64

In [18]:
NYPD_COMPL_df.vic_age_group.value_counts()

25-44      124364
UNKNOWN     74370
45-64       64090
18-24       31076
65+         13619
<18         10728
-948            2
-1              1
-963            1
-967            1
1016            1
-938            1
932             1
943             1
-31             1
949             1
938             1
-965            1
-968            1
-973            1
-60             1
-958            1
-2              1
Name: vic_age_group, dtype: int64

In [19]:
NYPD_COMPL_df.susp_age_group.value_counts()

UNKNOWN    144584
25-44      100238
45-64       33880
18-24       29613
<18          6617
65+          3312
2020           10
2019            2
-977            1
1020            1
-962            1
-942            1
-12             1
-71             1
-928            1
-965            1
1925            1
Name: susp_age_group, dtype: int64

In [20]:
age_group=["UNKNOWN","25-44","18-24","<18","65+","45-64"]
NYPD_COMPL_df=NYPD_COMPL_df[NYPD_COMPL_df['susp_age_group'].isin(age_group)]
NYPD_COMPL_df=NYPD_COMPL_df[NYPD_COMPL_df['vic_age_group'].isin(age_group)]
NYPD_COMPL_df.reset_index(inplace=True,drop=True)

In [21]:
NYPD_COMPL.vic_sex.value_counts()

F    164503
M    152219
D     63160
E     33529
Name: vic_sex, dtype: int64

In [22]:
NYPD_COMPL_df.susp_age_group.value_counts()

UNKNOWN    144577
25-44      100232
45-64       33877
18-24       29612
<18          6617
65+          3311
Name: susp_age_group, dtype: int64

In [23]:
NYPD_COMPL_df.ofns_desc.value_counts()

HARRASSMENT 2                           63150
PETIT LARCENY                           54898
ASSAULT 3 & RELATED OFFENSES            40806
CRIMINAL MISCHIEF & RELATED OF          27371
GRAND LARCENY                           21453
FELONY ASSAULT                          19003
OFF. AGNST PUB ORD SENSBLTY &           14263
ROBBERY                                 12617
MISCELLANEOUS PENAL LAW                 11753
BURGLARY                                10473
DANGEROUS DRUGS                          6562
OFFENSES AGAINST PUBLIC ADMINI           4994
SEX CRIMES                               4457
GRAND LARCENY OF MOTOR VEHICLE           4423
VEHICLE AND TRAFFIC LAWS                 3529
DANGEROUS WEAPONS                        2788
INTOXICATED & IMPAIRED DRIVING           1871
CRIMINAL TRESPASS                        1788
FORGERY                                  1616
RAPE                                     1392
THEFT-FRAUD                              1339
UNAUTHORIZED USE OF A VEHICLE     

In [24]:
NYPD_COMPL_df=NYPD_COMPL_df[NYPD_COMPL_df["cmplnt_fr_dt"]>="2020-01-01T00:00:00.000"]

In [25]:
NYPD_COMPL_df=NYPD_COMPL_df.drop_duplicates()

####   4.a Data cleansing on  New York State Statewide COVID-19 Testing

In [26]:
NYPD_COMPL_df.cmplnt_fr_dt.max()

'2020-12-31T00:00:00.000'

In [27]:
NYPD_COMPL_df

,cmplnt_fr_dt,boro_nm,law_cat_cd,ofns_desc,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
0,2020-11-22T00:00:00.000,BRONX,FELONY,RAPE,UNKNOWN,UNKNOWN,U,25-44,BLACK,F
1,2020-12-29T00:00:00.000,QUEENS,MISDEMEANOR,OFFENSES INVOLVING FRAUD,18-24,BLACK,M,UNKNOWN,UNKNOWN,E
2,2020-12-23T00:00:00.000,MANHATTAN,MISDEMEANOR,PETIT LARCENY,UNKNOWN,UNKNOWN,F,UNKNOWN,UNKNOWN,D
3,2020-12-31T00:00:00.000,MANHATTAN,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,UNKNOWN,UNKNOWN,U,UNKNOWN,UNKNOWN,E
4,2020-12-22T00:00:00.000,BROOKLYN,MISDEMEANOR,PETIT LARCENY,UNKNOWN,UNKNOWN,U,45-64,BLACK HISPANIC,M
...,...,...,...,...,...,...,...,...,...,...
318217,2020-01-02T00:00:00.000,BROOKLYN,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,UNKNOWN,BLACK HISPANIC,U,45-64,BLACK,F
318218,2020-01-01T00:00:00.000,MANHATTAN,FELONY,FELONY ASSAULT,25-44,BLACK,M,25-44,BLACK,M
318219,2020-01-05T00:00:00.000,BRONX,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,25-44,WHITE,M,25-44,WHITE HISPANIC,F
318222,2020-01-02T00:00:00.000,QUEENS,FELONY,CRIMINAL MISCHIEF & RELATED OF,UNKNOWN,UNKNOWN,U,45-64,ASIAN / PACIFIC ISLANDER,M


In [28]:
NYS_COV19_df.dropna(inplace=True)
NYS_COV19_df.shape

(25854, 4)

In [29]:
NYS_COV19_df.isna().sum()

test_date                0
county                   0
new_positives            0
total_number_of_tests    0
dtype: int64

In [30]:
NYS_COV19_df.head()

,test_date,county,new_positives,total_number_of_tests
0,2020-03-01T00:00:00.000,Albany,0,0
1,2020-03-02T00:00:00.000,Albany,0,0
2,2020-03-03T00:00:00.000,Albany,0,0
3,2020-03-04T00:00:00.000,Albany,0,0
4,2020-03-05T00:00:00.000,Albany,0,3


In [31]:
NYS_COV19_df.dtypes

test_date                object
county                   object
new_positives            object
total_number_of_tests    object
dtype: object

In [32]:
NYS_COV19_df["county"].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York', 'Niagara', 'Oneida', 'Onondaga', 'Ontario',
       'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam', 'Queens',
       'Rensselaer', 'Richmond', 'Rockland', 'Saratoga', 'Schenectady',
       'Schoharie', 'Schuyler', 'Seneca', 'St. Lawrence', 'Steuben',
       'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster', 'Warren',
       'Washington', 'Wayne', 'Westchester', 'Wyoming', 'Yates'],
      dtype=object)

In [33]:
NYC_list=["New York","Bronx","Richmond","Kings","Queens"]
NYC_COV19_df=NYS_COV19_df[NYS_COV19_df.county.isin(NYC_list)]
NYC_COV19_df.reset_index(inplace=True,drop=True)
NYC_COV19_df

,test_date,county,new_positives,total_number_of_tests
0,2020-03-01T00:00:00.000,Bronx,0,0
1,2020-03-02T00:00:00.000,Bronx,0,0
2,2020-03-03T00:00:00.000,Bronx,0,1
3,2020-03-04T00:00:00.000,Bronx,0,0
4,2020-03-05T00:00:00.000,Bronx,0,5
...,...,...,...,...
2080,2021-04-17T00:00:00.000,Richmond,250,7083
2081,2021-04-18T00:00:00.000,Richmond,206,4749
2082,2021-04-19T00:00:00.000,Richmond,187,3561
2083,2021-04-20T00:00:00.000,Richmond,161,5209


In [34]:
NYC_COV19_df.county.value_counts()

Bronx       417
Queens      417
Richmond    417
Kings       417
New York    417
Name: county, dtype: int64

In [35]:
boro_list = {"Bronx":'BRONX',"Queens":'QUEENS',"New York":'MANHATTAN',"Kings":'BROOKLYN',"Richmond":'STATEN ISLAND'}
NYC_COV19_df['county'] = [boro_list[item] for item in NYC_COV19_df['county']]

<ipython-input-35-cf01a19556b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYC_COV19_df['county'] = [boro_list[item] for item in NYC_COV19_df['county']]


## 5. Data Transforming (Dimension and Fact Table)

#### 5.a Create Suspect Dimension


In [36]:
suspect_df=NYPD_COMPL_df[["susp_age_group","susp_race","susp_sex"]]  
suspect_df=suspect_df.drop_duplicates()
Suspect_df=suspect_df.reset_index(drop=True)

In [37]:
suspect_df["suspect_id"]=list(range(100,100+len(suspect_df)))

In [38]:
suspect_dim=suspect_df[['suspect_id','susp_age_group', 'susp_race', 'susp_sex']]

In [39]:
suspect_dim

,suspect_id,susp_age_group,susp_race,susp_sex
0,100,UNKNOWN,UNKNOWN,U
1,101,18-24,BLACK,M
2,102,UNKNOWN,UNKNOWN,F
5,103,UNKNOWN,WHITE HISPANIC,M
6,104,25-44,WHITE,M
...,...,...,...,...
58535,218,<18,AMERICAN INDIAN/ALASKAN NATIVE,F
72933,219,65+,ASIAN / PACIFIC ISLANDER,U
89143,220,25-44,AMERICAN INDIAN/ALASKAN NATIVE,U
165221,221,65+,AMERICAN INDIAN/ALASKAN NATIVE,F


#### 5.b Create Victim Dimension


In [40]:
victim_df=NYPD_COMPL_df[["vic_age_group","vic_race","vic_sex"]]  
victim_df=victim_df.drop_duplicates()
victim_df=victim_df.reset_index(drop=True)

In [41]:
victim_df["victim_id"]=list(range(1000,1000+len(victim_df)))

In [42]:
victim_dim=victim_df[["victim_id","vic_age_group","vic_race","vic_sex"]]

In [43]:
victim_dim

,victim_id,vic_age_group,vic_race,vic_sex
0,1000,25-44,BLACK,F
1,1001,UNKNOWN,UNKNOWN,E
2,1002,UNKNOWN,UNKNOWN,D
3,1003,45-64,BLACK HISPANIC,M
4,1004,25-44,BLACK,M
...,...,...,...,...
113,1113,18-24,WHITE HISPANIC,E
114,1114,45-64,BLACK,D
115,1115,25-44,UNKNOWN,D
116,1116,18-24,BLACK,E


#### 5.c Create Offence Dimension


In [44]:
offence_df=NYPD_COMPL_df[["law_cat_cd","ofns_desc"]]  
offence_df=offence_df.drop_duplicates()
offence_df=offence_df.reset_index(drop=True)

In [45]:
offence_df["off_id"]=list(range(5000,5000+len(offence_df)))

In [46]:
offence_dim=offence_df[["off_id","law_cat_cd","ofns_desc"]]

In [47]:
offence_dim

,off_id,law_cat_cd,ofns_desc
0,5000,FELONY,RAPE
1,5001,MISDEMEANOR,OFFENSES INVOLVING FRAUD
2,5002,MISDEMEANOR,PETIT LARCENY
3,5003,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
4,5004,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
...,...,...,...
62,5062,MISDEMEANOR,ENDAN WELFARE INCOMP
63,5063,FELONY,ENDAN WELFARE INCOMP
64,5064,FELONY,GAMBLING
65,5065,MISDEMEANOR,OFFENSES RELATED TO CHILDREN


#### 5.d Create location Dimension

In [48]:
loc_list = NYPD_COMPL_df.boro_nm.unique().tolist()   
    # create blank list of dimension rows
dimension_rows = []
    
    # create author dimension with a surrogate key
for loc_id, boro_nm in enumerate(loc_list, start = 50):
    temp_list = [loc_id, boro_nm]
    dimension_rows.append(temp_list)

loc_dim = pd.DataFrame(data=dimension_rows,
                         columns = ["loc_id","borough"])

In [49]:
loc_dim

,loc_id,borough
0,50,BRONX
1,51,QUEENS
2,52,MANHATTAN
3,53,BROOKLYN
4,54,STATEN ISLAND


#### 5.e Create date Dimension

In [50]:
NYS_COV19_df.test_date.max()

'2021-04-21T00:00:00.000'

In [51]:
NYS_COV19_df.min()

test_date                2020-03-01T00:00:00.000
county                                    Albany
new_positives                                  0
total_number_of_tests                          0
dtype: object

In [52]:
NYS_COV19_df.test_date.str[0:10]

0        2020-03-01
1        2020-03-02
2        2020-03-03
3        2020-03-04
4        2020-03-05
            ...    
25849    2021-04-17
25850    2021-04-18
25851    2021-04-19
25852    2021-04-20
25853    2021-04-21
Name: test_date, Length: 25854, dtype: object

In [53]:
date_df=NYS_COV19_df[['test_date']]
date_df=date_df.rename(columns={"test_date":"full_date"})
date_df=date_df.drop_duplicates()
date_df2=NYPD_COMPL_df[['cmplnt_fr_dt']]
date_df2= date_df2.rename(columns={"cmplnt_fr_dt":"full_date"})
date_df2=date_df2.drop_duplicates()
date_df=date_df.append(date_df2)
date_df=date_df.drop_duplicates()

In [54]:
date_df['full_date']=date_df['full_date'].str[0:10]
date_df['date_id']=date_df['full_date'].str[0:4]+date_df['full_date'].str[5:7]+date_df['full_date'].str[8:10]
date_df['full_date']=pd.to_datetime(date_df['full_date'])
date_df["year"]=date_df['full_date'].dt.year
date_df["month"]=date_df['full_date'].dt.month
date_df["day"]=date_df['full_date'].dt.day
date_df["weekday"]=date_df['full_date'].dt.day_name()

In [55]:
date_df=date_df[["date_id","day","month","year","weekday"]]
date_df=date_df.sort_values("date_id")
date_dim=date_df.reset_index(drop=True)

In [56]:
date_dim

,date_id,day,month,year,weekday
0,20200101,1,1,2020,Wednesday
1,20200102,2,1,2020,Thursday
2,20200103,3,1,2020,Friday
3,20200104,4,1,2020,Saturday
4,20200105,5,1,2020,Sunday
...,...,...,...,...,...
472,20210417,17,4,2021,Saturday
473,20210418,18,4,2021,Sunday
474,20210419,19,4,2021,Monday
475,20210420,20,4,2021,Tuesday


#### 5.f Create covid-19 fact table

In [58]:
NYC_COV19_df1=NYC_COV19_df.copy()
NYC_COV19_df1['test_date']=NYC_COV19_df['test_date'].str[0:10]
NYC_COV19_df1['date_id']=NYC_COV19_df['test_date'].str[0:4]+NYC_COV19_df['test_date'].str[5:7]+NYC_COV19_df['test_date'].str[8:10]

In [59]:
for col in ["new_positives","total_number_of_tests"]:
    NYC_COV19_df1[col]=NYC_COV19_df1[col].astype('int64')
NYC_COV19_df1.dtypes

test_date                object
county                   object
new_positives             int64
total_number_of_tests     int64
date_id                  object
dtype: object

In [60]:
NYC_COV19_df1= NYC_COV19_df1.merge(date_dim, left_on='date_id', right_on='date_id',
                  how='inner')
NYC_COV19_df1= NYC_COV19_df1.merge(loc_dim, left_on='county', right_on='borough',
                  how='inner')
NYC_COV19_df1

,test_date,county,new_positives,total_number_of_tests,date_id,day,month,year,weekday,loc_id,borough
0,2020-03-01,BRONX,0,0,20200301,1,3,2020,Sunday,50,BRONX
1,2020-03-02,BRONX,0,0,20200302,2,3,2020,Monday,50,BRONX
2,2020-03-03,BRONX,0,1,20200303,3,3,2020,Tuesday,50,BRONX
3,2020-03-04,BRONX,0,0,20200304,4,3,2020,Wednesday,50,BRONX
4,2020-03-05,BRONX,0,5,20200305,5,3,2020,Thursday,50,BRONX
...,...,...,...,...,...,...,...,...,...,...,...
2080,2021-04-17,STATEN ISLAND,250,7083,20210417,17,4,2021,Saturday,54,STATEN ISLAND
2081,2021-04-18,STATEN ISLAND,206,4749,20210418,18,4,2021,Sunday,54,STATEN ISLAND
2082,2021-04-19,STATEN ISLAND,187,3561,20210419,19,4,2021,Monday,54,STATEN ISLAND
2083,2021-04-20,STATEN ISLAND,161,5209,20210420,20,4,2021,Tuesday,54,STATEN ISLAND


In [61]:
NYC_COV19_fact=NYC_COV19_df1.drop(["test_date","county","day","month","year","weekday","borough"], axis=1)

In [62]:
NYC_COV19_fact

,new_positives,total_number_of_tests,date_id,loc_id
0,0,0,20200301,50
1,0,0,20200302,50
2,0,1,20200303,50
3,0,0,20200304,50
4,0,5,20200305,50
...,...,...,...,...
2080,250,7083,20210417,54
2081,206,4749,20210418,54
2082,187,3561,20210419,54
2083,161,5209,20210420,54


#### 5.g Create crime fact table

In [63]:
NYPD_COMPL_df1=NYPD_COMPL_df.copy()
NYPD_COMPL_df1['cmplnt_fr_dt']=NYPD_COMPL_df1['cmplnt_fr_dt'].str[0:10]
NYPD_COMPL_df1['date_id']=NYPD_COMPL_df1['cmplnt_fr_dt'].str[0:4]+NYPD_COMPL_df1['cmplnt_fr_dt'].str[5:7]+NYPD_COMPL_df1['cmplnt_fr_dt'].str[8:10]

In [1]:
NYPD_COMPL_df1= NYPD_COMPL_df1.merge(date_dim, left_on='date_id', right_on='date_id',
                  how='inner')
NYPD_COMPL_df1= NYPD_COMPL_df1.merge(loc_dim, left_on='boro_nm', right_on='borough',
                  how='inner')
NYPD_COMPL_df1= NYPD_COMPL_df1.merge(offence_dim, left_on=["law_cat_cd","ofns_desc"], 
                                     right_on=["law_cat_cd","ofns_desc"],
                  how='inner')
NYPD_COMPL_df1= NYPD_COMPL_df1.merge(victim_dim, left_on=["vic_age_group","vic_race","vic_sex"], 
                                     right_on=["vic_age_group","vic_race","vic_sex"],
                  how='inner')
NYPD_COMPL_df1= NYPD_COMPL_df1.merge(suspect_dim, left_on=["susp_age_group","susp_race","susp_sex"], 
                                     right_on=["susp_age_group","susp_race","susp_sex"],
                  how='inner')
NYPD_COMPL_df1

NameError: name 'NYPD_COMPL_df1' is not defined

In [ ]:
NYPD_COMPL_fact=NYPD_COMPL_df1.drop(["cmplnt_fr_dt","boro_nm","law_cat_cd","ofns_desc","susp_age_group",
                                    "susp_race","susp_sex","vic_age_group","vic_race","vic_sex",
                                    "day","month","year","weekday","borough"],axis=1)

In [ ]:
NYPD_COMPL_fact

## 6.Load Dimensions and Fact Tables to BigQuery 

In [ ]:
key_path='crime-covid-178c3ff212a1.json'
# fill in file path to your key here
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [67]:
def load_df_to_bigquery(df, table_name):
    dataset_id =  'crime-covid:crime_covid_data'  
    dataset_ref = client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"
    
    upload_table_name = 'crime_covid_data.'+str(table_name)
    
    load_job = client.load_table_from_dataframe(df, upload_table_name,
                                                job_config=job_config)
    
    print("Starting job {}".format(load_job)) 

In [68]:
load_df_to_bigquery(df=date_dim, table_name='date_dim')
load_df_to_bigquery(df=loc_dim, table_name='loc_dim')
load_df_to_bigquery(df=offence_dim, table_name='offence_dim')
load_df_to_bigquery(df=victim_dim, table_name='victim_dim')
load_df_to_bigquery(df=suspect_dim, table_name='suspect_dim')
load_df_to_bigquery(df=NYC_COV19_fact, table_name='NYC_COV19_fact')
load_df_to_bigquery(df=NYPD_COMPL_fact, table_name='NYPD_COMPL_fact')

Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f099700>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f0993a0>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f4a4ac0>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f554490>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f554670>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f554a90>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f7f1f554e80>
